<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/IEEE%20paper%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autokeras


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.utils import get_custom_objects
from keras.layers import Activation
from keras.optimizers import Adam
import keras.backend as K

# Load Pima Indians Diabetes dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(url, names=names)

# Separate features and labels
X = data.drop('class', axis=1).values
Y = data['class'].values

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

def define_activation_functions():
    activations = [
        ('X_1', lambda x: K.pow(x, 1)),
        ('X_2', lambda x: K.pow(x, 2) / 2),
        ('X_2_', lambda x: K.pow(x, 2) / 24),
        ('X_2__', lambda x: K.pow(x, 2) / 720),
        ('X_2___', lambda x: K.pow(x, 2) / 40320),
    ]
    for name, func in activations:
        get_custom_objects().update({name: Activation(func)})

define_activation_functions()

def create_optimized_model(input_dim, hidden_dim, output_dim):
    input_layer = Input(shape=(input_dim,))
    x1 = Dense(hidden_dim, activation='X_1')(input_layer)
    x2 = Dense(hidden_dim, activation='X_2')(input_layer)
    merged1 = concatenate([x1, x2])
    x3 = Dense(hidden_dim, activation='X_2_')(merged1)
    x4 = Dense(hidden_dim, activation='X_2__')(x3)
    x5 = Dense(hidden_dim, activation='X_2___')(x4)
    merged2 = concatenate([x1, x2, x3, x4, x5])
    output = Dense(output_dim, activation='sigmoid')(merged2)
    model = Model(inputs=input_layer, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
    return model

input_dim = 8
hidden_dim = 50
output_dim = 1

# 10-fold cross-validation
n_folds = 10
kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=7)
cvscores = []

for train, test in kfold.split(X, Y):
    model = create_optimized_model(input_dim, hidden_dim, output_dim)
    model.fit(X[train], Y[train], epochs=10, batch_size=32, verbose=0)
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print(f"Fold Accuracy: {scores[1]*100:.2f}%")
    cvscores.append(scores[1] * 100)

print(f"{n_folds}-fold cross-validation average accuracy: {np.mean(cvscores):.2f}% (+/- {np.std(cvscores):.2f}%)")


Fold Accuracy: 71.43%
Fold Accuracy: 72.73%
Fold Accuracy: 80.52%
Fold Accuracy: 80.52%


Fold Accuracy: 68.83%


Fold Accuracy: 74.03%
Fold Accuracy: 77.92%
Fold Accuracy: 76.62%
Fold Accuracy: 73.68%
Fold Accuracy: 76.32%
10-fold cross-validation average accuracy: 75.26% (+/- 3.63%)


In [ ]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import autokeras as ak

# Load Pima Indians Diabetes dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(url, names=names)

# Separate features and labels
X = data.drop('class', axis=1).values
Y = data['class'].values

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 10-fold cross-validation
n_folds = 10
kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=7)
cvscores = []

for train, test in kfold.split(X, Y):
    # Use AutoKeras structured data classifier
    clf = ak.StructuredDataClassifier(overwrite=True, max_trials=5)
    clf.fit(X[train], Y[train], epochs=10, verbose=0)

    # Evaluate the model
    scores = clf.evaluate(X[test], Y[test], verbose=0)
    print(f"Fold Accuracy: {scores[1]*100:.2f}%")
    cvscores.append(scores[1] * 100)

print(f"{n_folds}-fold cross-validation average accuracy: {np.mean(cvscores):.2f}% (+/- {np.std(cvscores):.2f}%)")


Using TensorFlow backend
Fold Accuracy: 74.03%
Fold Accuracy: 72.73%
Fold Accuracy: 81.82%
Fold Accuracy: 79.22%
